In [ ]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute

cluster_name = "project"

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###
compute_config = AmlCompute.provisioning_configuration(
    vm_size = "Standard_D2_V2",
    max_nodes= 4
)

compute_target = ComputeTarget.create(ws, cluster_name, compute_config)



In [ ]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os

# Specify parameter sampler
ps = RandomParameterSampling({
    '--C': uniform(0.01, 1.0),
    '--max_iter': choice(25,50,100)})

# Specify a Policy
policy = BanditPolicy(evaluation_interval = 2, slack_factor= 0.1)

if "training" not in os.listdir():
    os.mkdir("./training")

# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

# Create a ScriptRunConfig Object to specify the configuration details of your training job
src = ScriptRunConfig(
    script = 'train.py',
    arguments = ['--C', 1.0, '--max_iter', 100.0],
    compute_target = compute_target,
    environment= sklearn_env
)

# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(
    hyperparameter_sampling= ps,
    policy = policy,
    primary_metric_name='Accuracy',
    primary_metric_goal= primary_metric_name.MAXIMIZE,
    max_total_runs=20.
    max_concurrent_runs=4
)

In [ ]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

hdr = exp.submit(config = hyperdrive_config)
RunDetails(hdr).show()

In [ ]:
import joblib
# Get your best run and save the model from that run.
best = hdr.get_best_run_by_primary_metric()

best.download_file('outputs/model.pkl', 'best_model.pkl')
model = joblib.load('best_model.pkl')



In [ ]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

ds = TabularDatasetFactory.from_delimited_files("https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv")

In [ ]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds)

train_df = x.copy()
train_df['y'] = y

In [ ]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task= 'classification',
    primary_metric= 'accuracy',
    training_data= train_df,
    label_column_name= "y",
    n_cross_validations= 3,
    featurization="auto",
    enable_early_stopping= TRUE,
    max_concurrent_iterations=4,
    max_cores_per_iteration= -1
    )

In [2]:
# Submit your automl run
automl_experiment = Experiment(workspace = ws, name = 'Project1')
automl_run = automl_experiment.submit(automl_config, show_output=False)

In [ ]:
# Retrieve and save your best automl model.

best_automl, fitted_model = automl_run.get_output()
os.makedirs("outputs", exist_ok=True)
joblib.dump(fitted_model, 'outputs/automl_model.pkl')

In [ ]:
# Deleting the Compute Cluster

compute_target.delete()